In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ase #package called Atomic Simulation Environment which allows us to construct crystals easily
from ase.spacegroup import crystal
from ase.visualize import view 
from ase.visualize.plot import plot_atoms
from tqdm import tqdm
from ase.io import read
from ase.io import write

In [ ]:
'''

This script is designed to generate a large scale crystal meant to import into a LAMMPS script using the ASE package. 
The input involves searching up a .cif file corresponding to the unit cell structure of that particularly
desired crystal. This unit cell can be multiplied many times into a much larger supercell structure. 

@author: kmream
'''

In [21]:
# Read CIF file -> these must be downloaded and located in the same repository 
#atoms = read('1011176.cif') #quartz
#atoms = read('9011998.cif') #silicon 

atoms2 = read('quartz_unitcell.cif') 

# Replicate the unit cell
atoms2 *= (75, 65, 65)

#view(atoms2) 


In [22]:
write("quartz_supercell2.lmp", atoms2, format="lammps-data", specorder=['Si', 'O'])

with open('quartz_supercell2.lmp', 'w') as f:
    f.write("ID Type x y z\n")
    for i, atom in enumerate(atoms2, start=1):
        atom_type = atom.number  # or use a mapping from symbol if needed
        x, y, z = atom.position
        f.write(f"{i} {atom_type} {x:.6f} {y:.6f} {z:.6f}\n")  

In [15]:
data = pd.read_csv('quartz_supercell2.txt', sep = " ")
data['ID'] = data.index + 14
data.loc[data['Type'] == 14, 'Type'] = 1
data.loc[data['Type'] == 8, 'Type'] = 2
charge_map = {1: 1.0, 2: -0.5}
data['charge'] = data['Type'].map(charge_map)
charge_col = data.pop('charge')
data.insert(2, 'charge', charge_col)
data

,ID,Type,charge,x,y,z
0,14,2,-0.5,1.374362,1.137040,4.245212
1,15,2,-0.5,3.241814,0.621712,2.443645
2,16,2,-0.5,2.754374,2.496637,0.642078
3,17,2,-0.5,0.297524,1.758752,1.159488
4,18,2,-0.5,-1.082488,3.118349,2.961055
...,...,...,...,...,...,...
2105995,2106009,2,-0.5,168.440162,254.186302,321.838355
2105996,2106010,2,-0.5,170.307614,254.701629,323.639922
2105997,2106011,1,1.0,171.830615,251.067953,322.480433
2105998,2106012,1,1.0,168.368668,253.066709,320.678867


In [ ]:
'''
if necessary, cut the data such that it is rectangular/perpendicular to each axes w.r.t. all 3 planes -> below code visualizes supercell crystal for verification
'''
# fig, axes = plt.subplots(1, 3, figsize=(15, 5), constrained_layout=True)

# axes[0].scatter(data['x'].values / 10, data['y'].values / 10, s = 2, marker = '.')
# axes[0].axvline(x = 0, color='r', linestyle='--')
# axes[0].axvline(x = 20, color='r', linestyle='--')
# axes[0].set_title('OG quartz imported')
# axes[0].set_xlabel('X (nanometers)')
# axes[0].set_ylabel('Y (nanometers)')

# axes[1].scatter(data['y'].values / 10, data['z'].values / 10, s = 2, marker = '.')
# axes[1].set_title('OG quartz imported')
# axes[1].set_xlabel('Y (nanometers)')
# axes[1].set_ylabel('Z (nanometers)')

# axes[2].scatter(data['x'].values / 10, data['z'].values / 10, s = 2, marker = '.')
# axes[2].set_title('OG quartz imported')
# axes[2].set_xlabel('X (nanometers)')
# axes[2].set_ylabel('Z (nanometers)')

In [ ]:
xcut, ycut, zcut = 275, 275, 275
def cutoff(data):
    data_new = data[(data['x'] < xcut)]
    data_new = data_new[data_new['y'] < ycut]
    data_new = data_new[data_new['z'] < zcut]
    return data_new

data = cutoff(data)

In [ ]:
def write_lammps_data(df, filename):
    with open(filename, 'w') as f:
        f.write("#LAMMPS data file via pandas\n\n")
        f.write(f"{len(df)} atoms\n")
        f.write(f"{df['Type'].nunique() + 1} atom types\n\n")

        # Assuming orthogonal box and bounds can be inferred from coordinates
        xlo, xhi = df['x'].min(), df['x'].max()
        ylo, yhi = df['y'].min(), df['y'].max()
        zlo, zhi = df['z'].min(), df['z'].max()

        # Add some padding on each wall
        pad = 3.0
        f.write(f"{xlo - pad:.6f} {xhi + pad:.6f} xlo xhi\n")
        #f.write(f"{ylo - pad:.6f} {yhi + pad:.6f} ylo yhi\n")
        f.write(f"-100 400 ylo yhi\n") #modify the direction which you're shooting the gold ion to be much longer 
        f.write(f"{zlo - pad:.6f} {zhi + pad:.6f} zlo zhi\n\n")
        
        f.write("Masses\n\n")
        f.write(f" 1 28.08500000 # Si\n")
        f.write(f" 2 15.99900000 # O\n")
        f.write(f" 3 196.9665500 # Au\n\n")

        f.write("Atoms\n\n")
        for _, row in df.iterrows():
            f.write(f"{int(row['ID'])} {int(row['Type'])} {int(row['charge'])} {row['x']} {row['y']} {row['z']}\n")

In [ ]:
write_lammps_data(data, 'quartz_supercell.lmp')